In [13]:
import tensorflow as tf

from tensorflow import keras

In [7]:
word_index = keras.datasets.imdb.get_word_index()
id_to_word = {id_ + 3: word for word, id_ in word_index.items()}
for id_, token in enumerate(("<pad>", "<sos>", "<unk>")):
  id_to_word[id_] = token

1654784/1641221 [==============================] - 0s 0us/step


In [10]:
" ".join([id_to_word[id_] for id_ in X_train[1][:20]])

'<sos> big hair big boobs bad music and a giant safety pin these are the words to best describe this'

In [3]:
(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data()

17473536/17464789 [==============================] - 0s 0us/step


In [5]:
import pandas as pd
pd.DataFrame(y_test).value_counts()

1    12500
0    12500
dtype: int64

In [6]:
pd.DataFrame(y_train).value_counts()

1    12500
0    12500
dtype: int64

In [11]:
import tensorflow_datasets as tfds
datasets, info = tfds.load("imdb_reviews", as_supervised=True,
with_info=True)
train_size = info.splits["train"].num_examples

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteS46Z70/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteS46Z70/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [14]:
def preprocess(X_batch, y_batch):
  X_batch = tf.strings.substr(X_batch, 0, 300)
  X_batch = tf.strings.regex_replace(X_batch, b"<br\\s*/?>", b" ")
  X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ")
  X_batch = tf.strings.split(X_batch)
  return X_batch.to_tensor(default_value=b"<pad>"), y_batch

In [15]:
from collections import Counter
vocabulary = Counter()
for X_batch, y_batch in datasets["train"].batch(32).map(preprocess):
  for review in X_batch:
    vocabulary.update(list(review.numpy()))

In [17]:
vocabulary.most_common()[:20]

[(b'<pad>', 214309),
 (b'the', 61137),
 (b'a', 38564),
 (b'of', 33983),
 (b'and', 33431),
 (b'to', 27707),
 (b'I', 27019),
 (b'is', 25719),
 (b'in', 18966),
 (b'this', 18490),
 (b'it', 17690),
 (b'was', 14950),
 (b'movie', 14945),
 (b'that', 14752),
 (b'The', 11879),
 (b'film', 10668),
 (b'with', 9073),
 (b'for', 8660),
 (b'as', 8578),
 (b'on', 7817)]

In [18]:
vocab_size = 10000
truncated_vocabulary = [word for word, count in vocabulary.most_common()[:vocab_size]]

In [19]:
words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

In [23]:
def encode_words(X_batch, y_batch):
  return table.lookup(X_batch), y_batch
train_set = datasets["train"].batch(32).map(preprocess)
test_set = datasets["test"].batch(32).map(preprocess)
train_set = train_set.map(encode_words).prefetch(1)
test_set =test_set.map(encode_words).prefetch(1)

In [26]:
embed_size = 128
model = keras.models.Sequential([
keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
input_shape=[None]),
keras.layers.GRU(128, return_sequences=True),
keras.layers.GRU(128),
keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam",
metrics=["accuracy"])
history = model.fit(train_set, epochs=5,validation_data=test_set)

Epoch 1/5
782/782 [==============================] - 177s 158ms/step - loss: 0.6804 - accuracy: 0.5470 - val_loss: 0.5943 - val_accuracy: 0.6838
Epoch 2/5
782/782 [==============================] - 123s 158ms/step - loss: 0.4572 - accuracy: 0.7812 - val_loss: 0.4762 - val_accuracy: 0.7711
Epoch 3/5
782/782 [==============================] - 124s 158ms/step - loss: 0.3073 - accuracy: 0.8734 - val_loss: 0.5549 - val_accuracy: 0.7528
Epoch 4/5
782/782 [==============================] - 124s 159ms/step - loss: 0.2106 - accuracy: 0.9223 - val_loss: 0.6689 - val_accuracy: 0.7408
Epoch 5/5
782/782 [==============================] - 124s 159ms/step - loss: 0.1515 - accuracy: 0.9470 - val_loss: 0.7729 - val_accuracy: 0.7347
